### Using FSMs to Lex in Project 1 ##

Let's implement the following FSM.

![colon-dash FSM](colon-dash_fsm_old.png)

### UPDATE ###

Let's explain the transitions and outputs. This FSM counts the number of characters it reads as long as the sequence of characters begins with ":-". The variable $o$ holds the number of characters it read. It is initialized to 0. If the input doesn't begin with ":-" then the output is set to 0. The $\lambda$ on the output means that the output doesn't change.

The input set $I$ contains any legal character that can appear in a program. Labeling an edge with $I$ indicates that the transition is taken for any input. Labeling an edge with $I-\{c\}$ means the edge is taken for any input except character $c$.

In [66]:
from typing import Callable as function
import my_token

# Define a type for State and the Output operator
State = function[[str], tuple[function, function]]
OutputOperator = function[[int], int]

class ColonDashFSM:
    def __init__(self) -> None:
        self.terminating_states: set[State] = {self.serr, self.sacc}

    #############################################
    ## Define a function that runs the machine ##
    #############################################
    def run(self, input_sequence: str) -> int:
        # Use a C++ style declaration of local variables
        present_state: State = self.s0
        next_state: State
        characters_counted: int = 0

        # Massage input so that it always has an extra character not in the pattern
        # This prevents finishing in something other than the accept or reject state
        input_sequence += "#"

        # Step through each input symbol
        for symbol in list(input_sequence):
            # Ask the present state what the next state and output should be
            next_state, output_operator = present_state(symbol)
            # Apply operator to output
            characters_counted = output_operator(characters_counted)
            # Update the state
            present_state = next_state
            # Don't keep running if you know you've succeeded or failed
            if present_state in self.terminating_states: break

        # return the number of characters counted
        return characters_counted

    ######################################
    ## Define a Function for each state ##
    ######################################
    def s0(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        match input_symbol:
            case ":": # Colon
                return (self.s1, lambda x: x+1)
            case _: # Anything else
                return (self.serr, lambda x: 0)
    
    def s1(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        match input_symbol:
            case "-": # Dash
                return (self.sacc, lambda x: x+1)
            case _: # Anything else
                return (self.serr, lambda x: 0)
    
    def serr(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        return (self.serr, lambda x: 0)

    def sacc(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        return (self.sacc, lambda x: x)

In [67]:
fsm: ColonDashFSM = ColonDashFSM()
def test_fsm(fsm):
    assert fsm.run(":-") == 2, "\':-\' failed"
    assert fsm.run(":") == 0, "\':\' failed"
    assert fsm.run(":---") == 2, "\':---\' failed"
    assert fsm.run("-:") == 0, "\'-:\' failed"
    print("All tests of the colon-dash fsm succeeded")

test_fsm(fsm)

All tests on colon-dash fsm succeeded


---

Let's repeat the exercise but with a FSM that counts whitespace. Recall that white space includes spaces (' '), tabs ('\\t'), carriage returns ('\\r'), and newlines ('\n'). Define the set $W = ${' ', '\\t\', '\\r', '\\n'}.

We can implement the following FSM that counts the whitespaces until it finds a character that is not a whitespace.

![whitespace FSM](whitespace_fsm.png)

In [68]:
from typing import Callable as function

# Define a type for State and the Output operator
State = function[[str], tuple[function, function]]
OutputOperator = function[[int], int]

class WhitespaceFSM:
    def __init__(self) -> None:
        self.terminating_states: set[State] = {self.serr, self.sacc}

    #############################################
    ## Define a function that runs the machine ##
    #############################################
    def run(self, input_sequence: str) -> int:
        # Use a C++ style declaration of local variables
        present_state: State = self.s0
        next_state: State
        characters_counted: int = 0

        # Massage input so that it always has an extra character not in the pattern
        # This prevents finishing in something other than the accept or reject state
        input_sequence += "#"

        # Step through each input symbol
        for symbol in list(input_sequence):
            # Ask the present state what the next state and output should be
            next_state, output_operator = present_state(symbol)
            # Apply operator to output
            characters_counted = output_operator(characters_counted)
            # Update the state
            present_state = next_state
            # Don't keep running if you know you've succeeded or failed
            if present_state in self.terminating_states: break

        # return the number of characters counted
        return characters_counted

    ######################################
    ## Define a Function for each state ##
    ######################################
    def s0(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        if input_symbol in {' ', '\t', '\r', '\n'}:
            return (self.s1, lambda x: x+1)
        else:
            return (self.serr, lambda x: 0)
    
    def s1(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        if input_symbol in {' ', '\t', '\r', '\n'}:
            return (self.s1, lambda x: x+1)
        else:
            return (self.sacc, lambda x: x)
    
    def serr(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        return (self.serr, lambda x: 0)

    def sacc(self, input_symbol: str) -> tuple[State, OutputOperator]:
        # return the next state and return the output as a tuple (next state, output operator)
        return (self.sacc, lambda x: x)

In [69]:
fsm: WhitespaceFSM = WhitespaceFSM()
def test_fsm(fsm):
    assert fsm.run(":-") == 0, "\':-\' failed"
    assert fsm.run(" \n\t\r") == 4, "\' \n\t\r' failed"
    assert fsm.run("\n\n hi \n\n") == 3, "\'\n\n hi \n\n\' failed"
    print("All tests of the whitespace fsm succeeded")

test_fsm(fsm)

All tests of the whitespace fsm succeeded


---

### Lexer Algorithm ###

The lexer algorithm takes the input string and tries to run every fsm on it. By design, each fsm only returns more than zero characters if the pattern in the input string is the one the fsm is looking for. The key idea of the lexer algorithm is to find the fsm that returns the most characters. When you've identified the fsm that returns the most characters then you know which token to produce.  

Let's step through this with a few examples. Suppose the input string is "(hello world)". We can visually inspect the input string and see that the sequence of tokens that should be produced when we lex this input string is
 - LEFT_PAREN
 - ID
 - WHITESPACE
 - RIGHT_PAREN

What is the output when we run the colon-dash fsm on the input string? The left parenthesis does not match the colon-dash pattern so it says "I successfully read 0 characters". The same thing happens when the colon fsm runs on the input string, the comma fsm, and the rules fsm. In fact, the only fsm that returns more than 0 characters is the left_paren fsm, which returns 1 character. We loop through all the fsms and remember which one successfully read the most characters.  We create a token associated with that fsm. We then remove the "(" from the input string and try again.

The lexing algorithm has the following steps

- while the input string still has characters to read
  - see which fsm reads the most characters, and call the number of characters $m$
  - create a token for that fsm
  - advance the input string $m$ characters so that we are now looking at the input after we've processed the token

Exactly one fsm returns more than one character except for the following cases
- Both the colon and colon-dash fsm read a character. This means that the pattern is ":-". Since the colon-dash machine reads two characters and the colon fsm reads only one, we say that the correct token is COLON-DASH because that is the correct pattern
- No fsm returns more than 0 characters. This means that whatever was in the input string didn't match any of our patterns. For example, the input string was "123". That input doesn't match anything. For this special case, we say that an undefined token was found and stop lexing.
- The input string is a keyword or begins with a keyword. For example,
  - The input string is "Rules".  Both the ID fsm and the Rules FSM say that five characters were read. It can't be an ID if it is a keyword, so we choose the Rules FSM. This can be done by making sure that the Rules FSM runs before the ID FSM. That way, when we do the comparison that selects the maximum value the Rules FSM wins.
  - The input string is "RulesRock". The Rules FSM returns five characters and the ID FSM returns 9. That means that the pattern is an ID, which is correct. We can have IDs that look a lot like keywords as long as they also include stuff at the end of the keyword.